# **PHÂN TÍCH NGỮ NGHĨA TƯỜNG MINH (Explicit Semantic Analysis)**

---

Phân tích ngữ nghĩa tường minh có đặc điểm sau:
- Tài liệu và truy vấn được biểu diễn theo vector đa chiều.
- Mỗi chiều ngữ nghĩa là một lớp văn bản, được xác định bằng một tập tài liệu văn bản được tuyển chọn trước.
- Giá trị mỗi chiều sẽ là giá trị của hàm phân lớp với tập các lớp đã được xác định.

**I) XÁC ĐỊNH TẬP CÁC LỚP**

***1) SỬ DỤNG NGỮ LIỆU PHÂN LỚP SẴN CÓ***
- Số chiều có thể ít.
- Một số ngữ liệu phân lớp văn bản của Kaggle: https://www.kaggle.com/datasets/guiyihan/text-classification-20?resource=download

***2) ÁP DỤNG PHƯƠNG PHÁP GOM CỤM***
- Số chiều N tùy chọn.
- Chỉ cần một tập ngữ liệu chưa gán nhãn.
- Vector hóa các file trong tập ngữ liệu.
- Xác định số phân lớp N chính là số chiều cần biểu diễn bằng phương pháp KMeans


In [ ]:
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
nltk.download('punkt')
nltk.download("stopwords")
stoplist = stopwords.words("english")

def getTerms(content):
  
  sents = sent_tokenize(content)
  words = []
  for sent in sents:
    words.extend(word_tokenize(sent))

  filtered = []
  for w in words:
    if w in stoplist:
      continue
    filtered.append(w)

  tokens = []
  for w in filtered:
    if w.isalnum():
      tokens.append(w)

  ps = PorterStemmer()
  terms = []
  for w in tokens:
    terms.append(ps.stem(w))
  return terms


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Ví dụ**

Cho tập văn bản D={d1, d2, d3, d4, d5}. Hãy xác định 3 lớp văn bản từ D.

d1 = Romeo and Juliet

d2 = Juliet: Oh happy dagger

d3 = Romeo died by dagger

d4 = "live free or die" is from New-Hampshire

d5 = He is from New-Hampshire


In [ ]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
D = ['Romeo and Juliet', "Juliet: Oh happy dagger", "Romeo died by dagger", '"live free or die" is from New-Hampshire', "He is from New-Hampshire"]


Vector hóa các tài liệu

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)

DD = []
for d in D:
  DD.append({"content":getTerms(d)})
print(DD)
vectors = dv.fit_transform(DD)
print(vectors)

[{'content': ['romeo', 'juliet']}, {'content': ['juliet', 'oh', 'happi', 'dagger']}, {'content': ['romeo', 'die', 'dagger']}, {'content': ['live', 'free', 'die']}, {'content': ['he']}]
[[0. 0. 0. 0. 0. 1. 0. 0. 1.]
 [1. 0. 0. 1. 0. 1. 0. 1. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 1. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]]


Thực hiện gom cụm

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
kmeans = KMeans(n_clusters=3, random_state=0).fit(vectors)
print(kmeans.labels_)
#kmeans.predict([[0, 0], [12, 3]])
print(kmeans.cluster_centers_)


[0 1 0 2 0]
[[0.33333333 0.33333333 0.         0.         0.33333333 0.33333333
  0.         0.         0.66666667]
 [1.         0.         0.         1.         0.         1.
  0.         1.         0.        ]
 [0.         1.         1.         0.         0.         0.
  1.         0.         0.        ]]


**II) HUẤN LUYỆN MÔ HÌNH PHÂN LỚP**

- Vector hóa các file trong tập ngữ liệu phân lớp.

- Xây dựng mô hình phân lớp đa lớp cho các vector này. Các mô hình có thể là SVM, Naive Bayes, KNN, ...

- Dự đoán giá trị phân lớp cho từng tài liệu và dùng kết quả phân lớp này như một vector biểu diễn của tài liệu đó.



In [ ]:
from sklearn.svm import SVC
classifier = SVC(C=2, kernel = "rbf",tol=0.5, probability=True)
classifier.fit(vectors,kmeans.labels_)


SVC(C=2, probability=True, tol=0.5)

**III) TÍNH VECTOR TÀI LIỆU VÀ TRUY VẤN**

- Xem truy vấn như một tài liệu, vector hóa tài liệu như trên.

- Tính xác suất của vector tài liệu hoặc truy vấn được phân vào các lớp.

- Các xác suất này được xem như là giá trị các chiều của vector tài liệu hoặc truy vấn.



In [ ]:
X = [0,0,1,0,0,0,0,0,1]
print(classifier.predict_proba([X]))

[[0.45324346 0.33084007 0.21591648]]


**BÀI TẬP**

Áp dụng phương pháp phân tích ngữ nghĩa tường minh cho ngữ liệu Cranfield và đánh giá kết quả truy vấn theo độ đo MAP 11 điểm nội suy.